# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [81]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [82]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,5.06,75,40,12.35,CL,1686424446
1,1,bilibino,68.0546,166.4372,17.17,39,10,1.52,RU,1686424446
2,2,saint-francois,46.4154,3.9054,21.69,73,0,1.54,FR,1686424446
3,3,port-aux-francais,-49.3500,70.2167,3.65,57,100,13.14,TF,1686424446
4,4,blackmans bay,-43.0167,147.3167,10.29,81,98,2.20,AU,1686424447


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [83]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City"    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [84]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=27)&
                                   (city_data_df["Max Temp"]>=20)&
                                   (city_data_df["Humidity"]<=75)&
                                   (city_data_df["Wind Speed"]<4.5)&
                                   (city_data_df["Cloudiness"]<=30),:]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna(how="any")

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,saint-francois,46.4154,3.9054,21.69,73,0,1.54,FR,1686424446
19,19,ribeira grande,38.5167,-28.7000,20.22,60,20,4.12,PT,1686424449
63,63,taos,36.4072,-105.5731,25.86,17,0,3.58,US,1686424455
77,77,winnemucca,40.9730,-117.7357,23.03,38,0,0.00,US,1686424456
101,101,bejaia,36.7559,5.0843,22.60,64,0,2.57,DZ,1686424459


### Step 3: Create a new DataFrame called `hotel_df`.

In [85]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,saint-francois,FR,46.4154,3.9054,73,
19,ribeira grande,PT,38.5167,-28.7000,60,
63,taos,US,36.4072,-105.5731,17,
77,winnemucca,US,40.9730,-117.7357,38,
101,bejaia,DZ,36.7559,5.0843,64,
127,leninskoye,KZ,41.7589,69.3864,45,
129,san juan bautista,PE,-14.0108,-75.7358,39,
162,balykshi,KZ,47.0667,51.8667,27,
164,grand baie,MU,-20.0182,57.5802,67,
180,laguna,US,38.4210,-121.4238,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [86]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
          "categories":categories,
          "apiKey":geoapify_key
          }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-francois - nearest hotel: Chez Lily
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
taos - nearest hotel: Hotel La Fonda de Taos
winnemucca - nearest hotel: The Overland Hotel
bejaia - nearest hotel: Hôtel du Nord
leninskoye - nearest hotel: No hotel found
san juan bautista - nearest hotel: Hacienda San Juan Resort & Spa
balykshi - nearest hotel: Арасан
grand baie - nearest hotel: Safari
laguna - nearest hotel: Holiday Inn Express & Suites
honefoss - nearest hotel: Comfort Hotel Ringerike
brookings - nearest hotel: Quality Inn
panjakent - nearest hotel: Umariyon
souillac - nearest hotel: Hotel SkylineInn
myrina - nearest hotel: Ήφαιστος
karratha - nearest hotel: Karratha International Hotel
belmonte - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: Hotel do Sol
emlichheim - nearest hotel: Pur Hotel
zakynthos - nearest hotel: Phoenix Hotel
lac-brome - nearest hotel: Auberge Lakeview Inn
rodez - nearest hotel: Auberge du Châ

,City,Country,Lat,Lng,Humidity,Hotel Name
2,saint-francois,FR,46.4154,3.9054,73,Chez Lily
19,ribeira grande,PT,38.5167,-28.7000,60,Hospedaria JSF - 3 Estrelas
63,taos,US,36.4072,-105.5731,17,Hotel La Fonda de Taos
77,winnemucca,US,40.9730,-117.7357,38,The Overland Hotel
101,bejaia,DZ,36.7559,5.0843,64,Hôtel du Nord
127,leninskoye,KZ,41.7589,69.3864,45,No hotel found
129,san juan bautista,PE,-14.0108,-75.7358,39,Hacienda San Juan Resort & Spa
162,balykshi,KZ,47.0667,51.8667,27,Арасан
164,grand baie,MU,-20.0182,57.5802,67,Safari
180,laguna,US,38.4210,-121.4238,56,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [87]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    alpha = .75,
    size = "Humidity",
    tiles = "OSM",
    hover = True,
    xlabel = "Latitude",
    ylabel = "Longitude",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Hotel Name","Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)